In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib notebook
# %config InlineBackend.figure_format = 'svg'

import os
import sys
sys.path.insert(0, os.path.dirname(os.path.dirname(os.getcwd())))
os.environ['THEANO_FLAGS'] = "device=cpu"

In [ ]:
import os
import shelve
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

plt.rcParams['savefig.dpi'] = 300

In [ ]:
from experiments.hmmvsrnn_reco.a_data import tmpdir, frame_seqs, dataset, get_ref_pts, detect_invalid_pts, interpolate_positions
from sltools.transform import Transformation, transform_pose2d, transform_pose3d, transform_frames

tgt_dist = 2
joints = dataset.JointType

flip_mapping = ([joints.ShoulderRight, joints.ElbowRight,
                 joints.WristRight, joints.HandRight, joints.ShoulderLeft,
                 joints.ElbowLeft, joints.WristLeft, joints.HandLeft,
                 joints.HipRight, joints.KneeRight, joints.AnkleRight,
                 joints.FootRight, joints.HipLeft, joints.KneeLeft,
                 joints.AnkleLeft, joints.FootLeft],
                [joints.ShoulderLeft, joints.ElbowLeft,
                 joints.WristLeft, joints.HandLeft, joints.ShoulderRight,
                 joints.ElbowRight, joints.WristRight, joints.HandRight,
                 joints.HipLeft, joints.KneeLeft, joints.AnkleLeft,
                 joints.FootLeft, joints.HipRight, joints.KneeRight,
                 joints.AnkleRight, joints.FootRight])

video = dataset.bgr_frames(0)
poses_2d = dataset.positions(0)
poses_3d = dataset.positions_3d(0)
invalid_masks = detect_invalid_pts(poses_2d)
poses_2d = interpolate_positions(poses_2d, invalid_masks)
poses_3d = interpolate_positions(poses_3d, invalid_masks)
ref2d = get_ref_pts(poses_2d)
ref3d = get_ref_pts(poses_3d)

zshifts = np.mean(tgt_dist - ref3d[:, 2])

In [ ]:
transformation = Transformation(
    ref2d=ref2d, ref3d=ref3d, flip_mapping=flip_mapping,
    frame_width=640,
    fliplr=False,
    tilt=5 * np.pi / 180,
    zshift=zshifts,
    xscale=1.15, yscale=0.85,
    zscale=1, tscale=1)

t = 10
plt.figure()
plt.imshow(video[t])
plt.scatter(poses_2d[t, :, 0], poses_2d[t, :, 1])
plt.figure()
plt.imshow(transform_frames(video, transformation)[t])
trans_pose2d = transform_pose2d(poses_2d, transformation)
plt.scatter(trans_pose2d[t, :, 0], trans_pose2d[t, :, 1])
plt.show()

In [ ]:
experiments = []

for run, directory in enumerate([tmpdir, tmpdir + '.run1']):
    for report_file in os.listdir(directory):
        if not report_file.endswith(".dat"):
            continue
        
        f = os.path.join(directory, report_file[:-4])
        with shelve.open(f, flag='r') as report:
            if 'analysis' not in report.keys():
                continue
            meta = report['meta']
            name = meta['experiment_name']
            args = report['args']['encoder_kwargs']
            analysis = report['analysis']
            experiments.append((name, run, meta, args, analysis))

In [ ]:
sorted([(e[0], e[4]['ji_filtered'][2]) for e in experiments])

# Varying TC size

In [ ]:
dtype = [('model', 'S3'), 
         ('win', 'i'),
         ('nparms', 'i'),
         ('ji', 'f'),
         ('acc', 'f')]

tc_analyses = np.sort([
    np.array((
        m['model'],
        a['filter_dilation'] * (a['tconv_sz'] - 1) + 1,
        a['tconv_sz'] * a['num_tc_filters'],
        r['ji_filtered'][2], 
        r['accuracy_filtered'][2]), 
        dtype=dtype)
    for _, _, m, a, r in experiments 
    if m['modality'] == "skel"])

plt.figure(dpi=100) 

legend = []
subset = (tc_analyses['model'] == b"rnn")
p1 = plt.scatter(
    tc_analyses[subset]['win'],
    tc_analyses[subset]['ji'],
    s=tc_analyses[subset]['nparms'] / 50,
    marker="o", alpha=0.5)
    
legend = []
subset = (tc_analyses['model'] == b"hmm")
p2 = plt.scatter(
    tc_analyses[subset]['win'],
    tc_analyses[subset]['ji'],
    s=tc_analyses[subset]['nparms'] / 50,
    marker="o", alpha=0.5)

plt.legend([p1, p2], ['rnn', 'hmm'], loc='best')

plt.xlabel("window size")
plt.ylabel("Jaccard Index")
plt.xticks(np.arange(3, 32, 4))
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)

plt.show()

# Transfer learning

In [ ]:
source_experiments = {
    (name, run): (meta['modality'], analysis['accuracy_filtered'][2], analysis['ji_filtered'][2])
    for name, run, meta, args, analysis in experiments 
    if meta['modality'] != "transfer"}

columns = [
    'model', 'modality', 'terminate_at', 'run', 
    'acc', 'delta_ref_acc', 'delta_other_acc', 'ji', 'delta_ref_ji', 'delta_other_ji']

transfer_analyses = []

for name, run, meta, args, analysis in experiments:
    if meta['modality'] != 'transfer':
        continue

    model = meta['model']
    terminate_at = args['terminate_at']
    modality, acc_other, ji_other = source_experiments[(args['transfer_from'], run)]
    _, acc_ref, ji_ref = source_experiments[(model + args['transfer_from'][3:], run)]
    acc = analysis['accuracy_filtered'][2]
    ji = analysis['ji_filtered'][2]
    transfer_analyses.append(
        (model, modality, terminate_at, run, 
                  acc, acc - acc_ref, acc - acc_other, 
                  ji, ji - ji_ref, ji - ji_other))

transfer_analyses = pd.DataFrame(transfer_analyses, columns=columns)

In [ ]:
transfer_analyses.groupby(['model', 'modality', 'terminate_at']).mean()

In [ ]:
# compare confusion on video frames

with shelve.open(os.path.join(tmpdir, "hmm_bgr_tc15"), flag='r') as report:
    hmm_meta = report['meta']
    hmm_args = report['args']
    hmm_analysis = report['analysis']
    
with shelve.open(os.path.join(tmpdir, "rnn_bgr_tc15"), flag='r') as report:
    rnn_meta = report['meta']
    rnn_args = report['args']
    rnn_analysis = report['analysis']

plt.figure(dpi=150)

bgr_conf = hmm_analysis['confusion_filtered'][1] / np.sum(hmm_analysis['confusion_filtered'][1], axis=1)
rnn_conf = rnn_analysis['confusion_filtered'][1] / np.sum(rnn_analysis['confusion_filtered'][1], axis=1)
conf_diff = bgr_conf - rnn_conf
limits = np.max(abs(conf_diff))
plt.imshow(conf_diff, clim=(-limits, limits), cmap='RdBu')
plt.yticks(np.arange(0, 21), [
    '∅','vattene','vieniqui','perfetto','furbo','cheduepalle','chevuoi','daccordo',
    'seipazzo','combinato','freganiente','ok','cosatifarei','basta','prendere',
    'noncenepiu','fame','tantotempo','buonissimo','messidaccordo','sonostufo'],
    fontsize=8)
plt.xticks(np.arange(0, 21), [''] * 21)
plt.gca().annotate(
    '', 
    xy=(1.4, 0.1), xycoords='axes fraction', xytext=(1.4, 0.9), 
    arrowprops=dict(arrowstyle="<->", color='k'))
plt.gca().annotate(
    'rnn', xy=(1.37, 0.05), xycoords='axes fraction', xytext=(1.37, 0.05))
plt.gca().annotate(
    'hmm', xy=(1.34, 0.92), xycoords='axes fraction', xytext=(1.34, 0.92))
plt.colorbar()

a = np.sum(hmm_analysis['confusion_filtered'][1][1:, 1:]) - np.sum(np.diag(hmm_analysis['confusion_filtered'][1][1:, 1:]))
b = np.sum(rnn_analysis['confusion_filtered'][1][1:, 1:]) - np.sum(np.diag(rnn_analysis['confusion_filtered'][1][1:, 1:]))
print(a, b, (a - b) / b)

In [ ]:
# compare confusion on body poses

with shelve.open(os.path.join(tmpdir, "hmm_skel_tc15"), flag='r') as report:
    hmm_meta = report['meta']
    hmm_args = report['args']
    hmm_analysis = report['analysis']
    
with shelve.open(os.path.join(tmpdir, "rnn_skel_tc15"), flag='r') as report:
    rnn_meta = report['meta']
    rnn_args = report['args']
    rnn_analysis = report['analysis']
    
plt.figure(dpi=150)

bgr_conf = hmm_analysis['confusion_filtered'][1] / np.sum(hmm_analysis['confusion_filtered'][1], axis=1)
rnn_conf = rnn_analysis['confusion_filtered'][1] / np.sum(rnn_analysis['confusion_filtered'][1], axis=1)
conf_diff = bgr_conf - rnn_conf
limits = np.max(abs(conf_diff))
plt.imshow(conf_diff, clim=(-limits, limits), cmap='RdBu')
plt.yticks(np.arange(0, 21), [
    '∅','vattene','vieniqui','perfetto','furbo','cheduepalle','chevuoi','daccordo',
    'seipazzo','combinato','freganiente','ok','cosatifarei','basta','prendere',
    'noncenepiu','fame','tantotempo','buonissimo','messidaccordo','sonostufo'],
    fontsize=8)
plt.xticks(np.arange(0, 21), [''] * 21)
plt.gca().annotate(
    '', 
    xy=(1.4, 0.1), xycoords='axes fraction', xytext=(1.4, 0.9), 
    arrowprops=dict(arrowstyle="<->", color='k'))
plt.gca().annotate(
    'rnn', xy=(1.37, 0.05), xycoords='axes fraction', xytext=(1.37, 0.05))
plt.gca().annotate(
    'hmm', xy=(1.34, 0.92), xycoords='axes fraction', xytext=(1.34, 0.92))
plt.colorbar()

a = np.sum(hmm_analysis['confusion_filtered'][1][1:, 1:]) - np.sum(np.diag(hmm_analysis['confusion_filtered'][1][1:, 1:]))
b = np.sum(rnn_analysis['confusion_filtered'][1][1:, 1:]) - np.sum(np.diag(rnn_analysis['confusion_filtered'][1][1:, 1:]))
print(a, b, (a - b) / b)

In [ ]:
# from sklearn.manifold import TSNE
# from sltools.tconv import TemporalConv

# tc_l = None
# layers = lasagne.layers.get_all_layers(model_dict['l_linout'])

# for l in layers:
#     if isinstance(l, TemporalConv):
#         tc_l = l
#         break

# W = np.asarray(tc_l.W.eval())
# tsne = TSNE(n_components=1, n_iter=5000, n_iter_without_progress=100, verbose=True)
# filter_order = np.argsort(tsne.fit_transform(W)[:, 0])